<a href="https://colab.research.google.com/github/SaikrishnaPulipati533/TERRABLUE_XT_PREDICT_EL_FUTURE/blob/master/terrablue_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import math

from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Activation, Dropout
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle

Using TensorFlow backend.


In [0]:
file= 'train_csv.csv'
data = pd.read_csv(file)
data.head()

,id,time,feature
0,0,2019-03-19 00:00:00,237.97
1,1,2019-03-19 00:00:10,57.93
2,2,2019-03-19 00:00:20,90.53
3,3,2019-03-19 00:00:30,260.75
4,4,2019-03-19 00:00:40,49.70


In [0]:
data.shape

(80, 3)

In [0]:
ts_data=data.iloc[:,2]
ts_data.head()

0    237.97
1     57.93
2     90.53
3    260.75
4     49.70
Name: feature, dtype: float64

In [0]:
scaler = MinMaxScaler(feature_range = (0, 1))
dataset = scaler.fit_transform(pd.DataFrame(ts_data))
# MinMaxScaler needs the data in 2D array , thats why the syntactical jugglery above

In [0]:
dataset.shape

(80, 1)

In [0]:
train_fraction = 0.60

train_size = int(len(dataset) * train_fraction)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size, :], dataset[train_size:len(dataset), :]

In [0]:
print(train_size, test_size)

48 32


In [0]:
def create_dataset(dataset, window_size = 1):
    data_X, data_Y = [], []
    for i in range(len(dataset) - window_size - 1):
        a = dataset[i:(i + window_size), 0]
        data_X.append(a)
        data_Y.append(dataset[i + window_size, 0])
    return(np.array(data_X), np.array(data_Y))

In [0]:
window_size = 10

train_X, train_Y = create_dataset(train, window_size)
test_X, test_Y = create_dataset(test, window_size)

print("Original training data shape:")
print(train_X.shape)

Original training data shape:
(37, 10)


In [0]:
range(len(dataset) - window_size - 1)

range(0, 69)

In [0]:
pd.DataFrame(train_X).head()

,0,1,2,3,4,5,6,7,8,9
0,0.483116,0.081564,0.154273,0.533924,0.063208,0.626104,0.291931,0.044228,0.029552,0.411522
1,0.081564,0.154273,0.533924,0.063208,0.626104,0.291931,0.044228,0.029552,0.411522,0.323111
2,0.154273,0.533924,0.063208,0.626104,0.291931,0.044228,0.029552,0.411522,0.323111,0.116848
3,0.533924,0.063208,0.626104,0.291931,0.044228,0.029552,0.411522,0.323111,0.116848,0.582345
4,0.063208,0.626104,0.291931,0.044228,0.029552,0.411522,0.323111,0.116848,0.582345,0.226046


In [0]:
# shape of data should be : (num_obs, feature_dimension, sequence_length)
train_X = np.reshape(train_X, (train_X.shape[0], 1, train_X.shape[1]))
test_X = np.reshape(test_X, (test_X.shape[0], 1, test_X.shape[1]))

print("New training data shape:")
print(train_X.shape)

New training data shape:
(37, 1, 10)


In [0]:
# input shape : (feature_dimension, seq_length)
model = Sequential()
    
model.add(LSTM(4, 
               input_shape = (1, window_size)))
model.add(Dense(1))
model.compile(loss = "mean_squared_error", 
              optimizer = "adam")

model.fit(train_X, 
          train_Y, 
          epochs = 100, 
          batch_size = 1, 
          verbose = 2)

Epoch 1/100
 - 1s - loss: 0.1226
Epoch 2/100
 - 0s - loss: 0.0876
Epoch 3/100
 - 0s - loss: 0.0667
Epoch 4/100
 - 0s - loss: 0.0540
Epoch 5/100
 - 0s - loss: 0.0453
Epoch 6/100
 - 0s - loss: 0.0406
Epoch 7/100
 - 0s - loss: 0.0378
Epoch 8/100
 - 0s - loss: 0.0367
Epoch 9/100
 - 0s - loss: 0.0355
Epoch 10/100
 - 0s - loss: 0.0347
Epoch 11/100
 - 0s - loss: 0.0340
Epoch 12/100
 - 0s - loss: 0.0335
Epoch 13/100
 - 0s - loss: 0.0332
Epoch 14/100
 - 0s - loss: 0.0327
Epoch 15/100
 - 0s - loss: 0.0323
Epoch 16/100
 - 0s - loss: 0.0317
Epoch 17/100
 - 0s - loss: 0.0313
Epoch 18/100
 - 0s - loss: 0.0308
Epoch 19/100
 - 0s - loss: 0.0304
Epoch 20/100
 - 0s - loss: 0.0299
Epoch 21/100
 - 0s - loss: 0.0297
Epoch 22/100
 - 0s - loss: 0.0292
Epoch 23/100
 - 0s - loss: 0.0288
Epoch 24/100
 - 0s - loss: 0.0284
Epoch 25/100
 - 0s - loss: 0.0278
Epoch 26/100
 - 0s - loss: 0.0275
Epoch 27/100
 - 0s - loss: 0.0273
Epoch 28/100
 - 0s - loss: 0.0267
Epoch 29/100
 - 0s - loss: 0.0267
Epoch 30/100
 - 0s - lo

In [0]:
# Make predictions on the original scale of the data.
pred = scaler.inverse_transform(model.predict(train_X))

# Prepare Y data to also be on the original scale for interpretability.
orig_data = scaler.inverse_transform([train_Y])

# Calculate RMSE.
score_test = np.sqrt(mean_squared_error(orig_data[0], pred[:, 0]))

In [0]:
orig_data[0]

array([166.23,  73.75, 282.46, 122.71,  32.04,  40.73,  55.06,  46.4 ,
        54.46,  29.97,  40.43,  21.36,  46.59,  43.11,  55.62,  27.36,
        50.94, 116.61,  91.8 , 198.32, 326.76, 266.69, 205.95, 149.42,
       233.22,  63.63,  89.77, 255.79, 161.08, 123.41, 239.69, 108.12,
        94.31, 224.3 , 184.75, 113.74, 146.06])

In [0]:
#print("Training data score: ",  score_train)
print("Test data score: ",  score_test)

Test data score:  53.16491659707865
